# Agrupación y Análisis de Datos con GroupBy

**Curso:** Python para Ciencia de Datos \
**Fecha:** 08 de octubre de 2025 \
**Ruta:** Fundamentos de Data Science e IA \
**Repositorio:** `bootcamp-fundamentos-ciencia-de-datos` 

## 📋 Índice

1. [Concepto](#concepto)
2. [Distribución de Frecuencias](#distribucion-frecuencias)
3. [Agrupación Básica con groupby()](#agrupacion-basica)
4. [Múltiples Agregaciones con agg()](#multiples-agregaciones)
5. [Agrupación por Múltiples Columnas](#agrupacion-multiple)
6. [Funciones Personalizadas con apply()](#funciones-personalizadas)
7. [Casos de Uso en Data Science](#casos-uso)
8. [Recursos Adicionales](#recursos)

## Concepto

**GroupBy** es una de las operaciones más poderosas en Pandas. Permite dividir un DataFrame en grupos basados en valores de una o más columnas, aplicar funciones de agregación y combinar los resultados.

### El paradigma Split-Apply-Combine

DataFrame Original \
↓ \
[SPLIT] - Dividir en grupos \
↓ \
[APPLY] - Aplicar función a cada grupo \
↓ \
[COMBINE] - Combinar resultados \
↓ \
Resultado Agregado

### ¿Por qué es importante?

- 📊 **Análisis por categorías**: Ventas por país, región, producto
- 📈 **Cálculo de métricas**: Promedios, totales, conteos por grupo
- 🎯 **Identificar patrones**: Top productos, clientes VIP, tendencias
- 💡 **Toma de decisiones**: Comparar rendimiento entre segmentos

### Funciones de agregación comunes

| Función | Descripción | Ejemplo |
|---------|-------------|---------|
| `sum()` | Suma de valores | Ventas totales por país |
| `mean()` | Promedio | Ticket promedio por tienda |
| `count()` | Conteo de registros | Número de transacciones |
| `min()` / `max()` | Valores extremos | Precio más bajo/alto |
| `std()` | Desviación estándar | Variabilidad de precios |
| `agg()` | Múltiples agregaciones | Suma + Media + Conteo |

In [51]:
from data_loader import load_data
import pandas as pd
import numpy as np

In [52]:
# Cargar datos de ejemplo
df = load_data('online_retail.csv')

# Crear columna Total_Price si no existe
if 'Total_Price' not in df.columns:
    df['Total_Price'] = df['Quantity'] * df['UnitPrice']

print("Dataset cargado:")
print(df.head())
print(f"\nForma: {df.shape}")

📊 Separador detectado: ','
🔤 Encoding detectado: ascii (100.0% confianza)
⚠️ Error de encoding, intentando con latin-1
Dataset cargado:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

    InvoiceDate  UnitPrice  CustomerID         Country  Total_Price  
0  12/1/10 8:26       2.55     17850.0  United Kingdom        15.30  
1  12/1/10 8:26       3.39     17850.0  United Kingdom        20.34  
2  12/1/10 8:26       2.75     17850.0  United Kingdom        22.00  
3  12/1/10 8:26       3.39     17850.0  United Kingdom        20.34  
4  12/1/10 8:26       3.39     17850.0  United Kingdom        20.34  

Forma: (54

---

## Distribución de Frecuencias

Antes de agrupar, es útil entender **cuántas veces aparece cada valor** en una columna.

### value_counts() - Contar ocurrencias

El método `value_counts()` es perfecto para obtener la frecuencia de cada valor único.

In [53]:
# Contar ventas por país
country_counts = df['Country'].value_counts()

print("📊 Número de transacciones por país:")
print(country_counts.head(10))

📊 Número de transacciones por país:
Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: count, dtype: int64


### Análisis de la distribución

In [54]:
# Estadísticas de la distribución
print(f"🌍 Total de países: {df['Country'].nunique()}")
print(f"📦 Total de transacciones: {len(df)}")
print(f"\n🏆 País con más transacciones: {country_counts.index[0]}")
print(f"📈 Cantidad: {country_counts.iloc[0]:,}")

# Porcentaje del top país
porcentaje_top = (country_counts.iloc[0] / len(df)) * 100
print(f"📊 Representa el {porcentaje_top:.2f}% del total")

🌍 Total de países: 38
📦 Total de transacciones: 541909

🏆 País con más transacciones: United Kingdom
📈 Cantidad: 495,478
📊 Representa el 91.43% del total


### Visualizar top y bottom países

In [55]:
# Top 5 y Bottom 5
print("🥇 Top 5 países por número de transacciones:")
print(country_counts.head())

print("\n📉 Bottom 5 países por número de transacciones:")
print(country_counts.tail())

🥇 Top 5 países por número de transacciones:
Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Name: count, dtype: int64

📉 Bottom 5 países por número de transacciones:
Country
Lithuania         35
Brazil            32
Czech Republic    30
Bahrain           19
Saudi Arabia      10
Name: count, dtype: int64


---

## Agrupación Básica con groupby()

`groupby()` agrupa filas que tienen el mismo valor en una o más columnas.

### Sintaxis básica
```python
df.groupby('columna')['columna_a_agregar'].funcion_agregacion()

In [56]:
# Agrupar por país y sumar las cantidades
country_quantity = df.groupby('Country')['Quantity'].sum()

print("📦 Cantidad total de productos vendidos por país:")
print(country_quantity.sort_values(ascending=False).head(10))

📦 Cantidad total de productos vendidos por país:
Country
United Kingdom    4263829
Netherlands        200128
EIRE               142637
Germany            117448
France             110480
Australia           83653
Sweden              35637
Switzerland         30325
Spain               26824
Japan               25218
Name: Quantity, dtype: int64


### Ejemplo 2: Promedio de precios por país

In [44]:
# Precio unitario promedio por país
country_avg_price = df.groupby('Country')['UnitPrice'].mean()

print("💰 Precio unitario promedio por país:")
print(country_avg_price.sort_values(ascending=False).head(10))

💰 Precio unitario promedio por país:
Country
Singapore    109.645808
Hong Kong     42.505208
Portugal       8.582976
Cyprus         6.302363
Canada         6.030331
Norway         6.012026
EIRE           5.911077
Finland        5.448705
Lebanon        5.387556
Malta          5.244173
Name: UnitPrice, dtype: float64


### Ejemplo 3: Conteo de transacciones con count()

In [57]:
# Contar número de transacciones por país
country_transactions = df.groupby('Country')['InvoiceNo'].count()

print("🧾 Número de transacciones por país:")
print(country_transactions.sort_values(ascending=False).head(10))

🧾 Número de transacciones por país:
Country
United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: InvoiceNo, dtype: int64


### Comparación: value_counts() vs groupby().count()

Ambos cuentan ocurrencias, pero tienen diferencias:

| Aspecto | value_counts() | groupby().count() |
|---------|----------------|-------------------|
| **Uso** | Contar valores en UNA columna | Agrupar y contar en CUALQUIER columna |
| **Resultado** | Series ordenada por frecuencia | Series/DataFrame sin orden |
| **Flexibilidad** | Simple y directo | Más flexible, permite más operaciones |
| **Valores NaN** | Excluye por defecto | Excluye por defecto |

---

## Múltiples Agregaciones con agg()

`agg()` permite aplicar **múltiples funciones de agregación** a la vez.

### Sintaxis
```python
df.groupby('columna')['columna_numerica'].agg(['funcion1', 'funcion2', ...])

In [46]:
# Múltiples estadísticas del precio unitario por país
country_price_stats = df.groupby('Country')['UnitPrice'].agg(['sum', 'mean', 'min', 'max', 'std'])

print("📊 Estadísticas de precio unitario por país:")
print(country_price_stats.sort_values('sum', ascending=False).head(10))

📊 Estadísticas de precio unitario por país:
                        sum        mean       min       max         std
Country                                                                
United Kingdom  2245715.474    4.532422 -11062.06  38970.00   99.315438
EIRE              48447.190    5.911077      0.00   1917.00   54.035173
France            43031.990    5.028864      0.00   4161.06   79.909126
Germany           37666.000    3.966930      0.00    599.50   16.549026
Singapore         25108.890  109.645808      0.19   3949.32  515.275500
Portugal          13037.540    8.582976      0.12   1241.98   71.379996
Spain             12633.450    4.987544      0.00   1715.85   41.003162
Hong Kong         12241.500   42.505208      0.21   2653.95  307.646598
Belgium            7540.130    3.644335      0.12     39.95    4.244522
Switzerland        6813.690    3.403442      0.00     40.00    5.389581


### Renombrar columnas en agg()

Podemos dar nombres personalizados a las columnas resultantes:

In [47]:
# Estadísticas con nombres personalizados
country_stats_custom = df.groupby('Country')['Quantity'].agg([
    ('Total_Items', 'sum'),
    ('Avg_Items_Per_Transaction', 'mean'),
    ('Min_Items', 'min'),
    ('Max_Items', 'max'),
    ('Num_Transactions', 'count')
])

print("📦 Estadísticas de cantidad por país:")
print(country_stats_custom.sort_values('Total_Items', ascending=False).head(10))

📦 Estadísticas de cantidad por país:
                Total_Items  Avg_Items_Per_Transaction  Min_Items  Max_Items  \
Country                                                                        
United Kingdom      4263829                   8.605486     -80995      80995   
Netherlands          200128                  84.406580       -480       2400   
EIRE                 142637                  17.403245       -288       1440   
Germany              117448                  12.369458       -288        600   
France               110480                  12.911067       -250        912   
Australia             83653                  66.444003       -120       1152   
Sweden                35637                  77.136364       -240        768   
Switzerland           30325                  15.147353       -120        288   
Spain                 26824                  10.589814       -288        360   
Japan                 25218                  70.441341       -624       2040   

  

### Agregar múltiples columnas a la vez

In [58]:
# Diferentes agregaciones para diferentes columnas
country_summary = df.groupby('Country').agg({
    'InvoiceNo': 'nunique',      # Número de facturas únicas
    'Quantity': ['sum', 'mean'],  # Total y promedio de cantidad
    'UnitPrice': 'mean',          # Precio promedio
    'Total_Price': 'sum'          # Ingresos totales
})

print("📊 Resumen completo por país:")
print(country_summary.sort_values(('Total_Price', 'sum'), ascending=False).head(10))

📊 Resumen completo por país:
               InvoiceNo Quantity            UnitPrice  Total_Price
                 nunique      sum       mean      mean          sum
Country                                                            
United Kingdom     23494  4263829   8.605486  4.532422  8187806.364
Netherlands          101   200128  84.406580  2.738317   284661.540
EIRE                 360   142637  17.403245  5.911077   263276.820
Germany              603   117448  12.369458  3.966930   221698.210
France               461   110480  12.911067  5.028864   197403.900
Australia             69    83653  66.444003  3.220612   137077.270
Switzerland           74    30325  15.147353  3.403442    56385.350
Spain                105    26824  10.589814  4.987544    54774.580
Belgium              119    23152  11.189947  3.644335    40910.960
Sweden                46    35637  77.136364  3.910887    36595.910


### Aplanar columnas multi-nivel

Cuando usamos `agg()` con múltiples funciones, obtenemos columnas multi-nivel. Podemos aplanarlas:

In [49]:
# Aplanar las columnas
country_summary_flat = country_summary.copy()
country_summary_flat.columns = ['_'.join(col).strip() if col[1] else col[0]
                                 for col in country_summary_flat.columns.values]
country_summary_flat = country_summary_flat.reset_index()

print("📊 Resumen con columnas planas:")
print(country_summary_flat.head())

📊 Resumen con columnas planas:
     Country  InvoiceNo_nunique  Quantity_sum  Quantity_mean  UnitPrice_mean  \
0  Australia                 69         83653      66.444003        3.220612   
1    Austria                 19          4827      12.037406        4.243192   
2    Bahrain                  4           260      13.684211        4.556316   
3    Belgium                119         23152      11.189947        3.644335   
4     Brazil                  1           356      11.125000        4.456250   

   Total_Price_sum  
0        137077.27  
1         10154.32  
2           548.40  
3         40910.96  
4          1143.60  


---

## Agrupación por Múltiples Columnas

A veces necesitamos agrupar por **más de una columna** para análisis más granulares.

### Sintaxis
```python
df.groupby(['columna1', 'columna2'])['columna_numerica'].funcion()

In [59]:
# Agrupar por país y código de producto
country_product_sales = df.groupby(['Country', 'StockCode'])['Quantity'].sum()

print("📦 Ventas por país y producto (primeros 20):")
print(country_product_sales.sort_values(ascending=False).head(20))

📦 Ventas por país y producto (primeros 20):
Country         StockCode
United Kingdom  22197        52928
                84077        48326
                85099B       43167
                85123A       36706
                84879        33519
                22616        25307
                21212        24702
                22178        23242
                17003        22801
                21977        20288
                15036        19792
                22386        18936
                84946        18684
                22086        18197
                21915        17954
                23203        17478
                22469        17349
                47566        16709
                22355        15975
                84755        15400
Name: Quantity, dtype: int64


### Ejemplo 2: Convertir a DataFrame con reset_index()

El resultado anterior es una Series con índice multi-nivel. Para trabajar mejor con él:

In [60]:
# Convertir a DataFrame
country_product_df = df.groupby(['Country', 'StockCode']).agg({
    'Quantity': 'sum',
    'Total_Price': 'sum',
    'InvoiceNo': 'nunique'
}).reset_index()

# Renombrar columnas
country_product_df.columns = ['Country', 'StockCode', 'Total_Quantity', 'Total_Revenue', 'Num_Orders']

print("📊 DataFrame de ventas por país y producto:")
print(country_product_df.sort_values('Total_Revenue', ascending=False).head(15))

📊 DataFrame de ventas por país y producto:
              Country StockCode  Total_Quantity  Total_Revenue  Num_Orders
19484  United Kingdom       DOT            1707      206245.48         710
16776  United Kingdom     22423           10323      134405.94        1835
18161  United Kingdom     47566           16709       92501.73        1613
18962  United Kingdom    85123A           36706       92179.10        2150
18941  United Kingdom    85099B           43167       84516.44        1978
16471  United Kingdom     22086           18197       61888.19        1134
18732  United Kingdom     84879           33519       54662.15        1383
18352  United Kingdom     79321           10191       52986.86         662
16850  United Kingdom     22502            1712       50218.42         466
16575  United Kingdom     22197           52928       48214.77        1340
15832  United Kingdom     21137           11268       39387.00         363
17410  United Kingdom     23084           15168       378

### Ejemplo 3: Análisis por país y mes

In [61]:
# Asegurar que InvoiceDate es datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extraer año y mes
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month

# Agrupar por país, año y mes
country_month_sales = df.groupby(['Country', 'Year', 'Month']).agg({
    'Total_Price': 'sum',
    'InvoiceNo': 'nunique'
}).reset_index()

country_month_sales.columns = ['Country', 'Year', 'Month', 'Revenue', 'Num_Invoices']

print("📅 Ventas mensuales por país:")
print(country_month_sales[country_month_sales['Country'] == 'United Kingdom'].head(10))

C:\Users\hamtr\AppData\Local\Temp\ipykernel_17340\3013738927.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


📅 Ventas mensuales por país:
            Country  Year  Month     Revenue  Num_Invoices
294  United Kingdom  2010     12  676742.620          1885
295  United Kingdom  2011      1  434308.300          1327
296  United Kingdom  2011      2  408247.910          1259
297  United Kingdom  2011      3  559707.390          1802
298  United Kingdom  2011      4  442254.041          1622
299  United Kingdom  2011      5  596459.860          1973
300  United Kingdom  2011      6  554478.350          1830
301  United Kingdom  2011      7  565479.841          1764
302  United Kingdom  2011      8  539130.500          1546
303  United Kingdom  2011      9  862018.152          2090


---

## Funciones Personalizadas con apply()

Cuando las funciones de agregación estándar no son suficientes, usamos `apply()` con funciones personalizadas.

### Sintaxis
```python
df.groupby('columna').apply(funcion_personalizada)

In [62]:

def calculate_total_revenue(group):
    """
    Calcula el ingreso total para un grupo de transacciones.

    Args:
        group: DataFrame con un grupo de filas

    Returns:
        float: Ingreso total del grupo
    """
    return (group['Quantity'] * group['UnitPrice']).sum()

# Aplicar la función a cada país
country_revenue = df.groupby('Country').apply(calculate_total_revenue)
country_revenue = country_revenue.sort_values(ascending=False)

print("💰 Ingresos totales por país:")
print(country_revenue.head(10))

💰 Ingresos totales por país:
Country
United Kingdom    8187806.364
Netherlands        284661.540
EIRE               263276.820
Germany            221698.210
France             197403.900
Australia          137077.270
Switzerland         56385.350
Spain               54774.580
Belgium             40910.960
Sweden              36595.910
dtype: float64


C:\Users\hamtr\AppData\Local\Temp\ipykernel_17340\22740754.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  country_revenue = df.groupby('Country').apply(calculate_total_revenue)


### Comparación: apply() vs operación directa

En el ejemplo anterior, podríamos haberlo hecho más simple:

In [63]:
# Forma más simple (recomendada cuando es posible)
country_revenue_simple = df.groupby('Country')['Total_Price'].sum().sort_values(ascending=False)

print("💰 Ingresos totales por país (método simple):")
print(country_revenue_simple.head(10))

# Verificar que dan el mismo resultado
print(f"\n¿Son iguales? {country_revenue.equals(country_revenue_simple)}")

💰 Ingresos totales por país (método simple):
Country
United Kingdom    8187806.364
Netherlands        284661.540
EIRE               263276.820
Germany            221698.210
France             197403.900
Australia          137077.270
Switzerland         56385.350
Spain               54774.580
Belgium             40910.960
Sweden              36595.910
Name: Total_Price, dtype: float64

¿Son iguales? False


### Ejemplo 2: Función personalizada que retorna múltiples valores

In [64]:
def customer_metrics(group):
    """
    Calcula métricas completas por grupo.

    Returns:
        Series: Múltiples métricas calculadas
    """
    return pd.Series({
        'Total_Revenue': group['Total_Price'].sum(),
        'Avg_Transaction': group['Total_Price'].mean(),
        'Num_Transactions': len(group),
        'Num_Items': group['Quantity'].sum(),
        'Avg_Items_Per_Transaction': group['Quantity'].mean()
    })

# Aplicar función que retorna múltiples valores
country_metrics = df.groupby('Country').apply(customer_metrics)

print("📊 Métricas completas por país:")
print(country_metrics.sort_values('Total_Revenue', ascending=False).head(10))

📊 Métricas completas por país:
                Total_Revenue  Avg_Transaction  Num_Transactions  Num_Items  \
Country                                                                       
United Kingdom    8187806.364        16.525065          495478.0  4263829.0   
Netherlands        284661.540       120.059696            2371.0   200128.0   
EIRE               263276.820        32.122599            8196.0   142637.0   
Germany            221698.210        23.348943            9495.0   117448.0   
France             197403.900        23.069288            8557.0   110480.0   
Australia          137077.270       108.877895            1259.0    83653.0   
Switzerland         56385.350        28.164510            2002.0    30325.0   
Spain               54774.580        21.624390            2533.0    26824.0   
Belgium             40910.960        19.773301            2069.0    23152.0   
Sweden              36595.910        79.211926             462.0    35637.0   

                Avg_

C:\Users\hamtr\AppData\Local\Temp\ipykernel_17340\2134431360.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  country_metrics = df.groupby('Country').apply(customer_metrics)


### Ejemplo 3: Top 3 productos por país

In [65]:
def top_3_products(group):
    """
    Retorna los 3 productos más vendidos en el grupo.
    """
    top_products = group.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(3)
    return top_products

# Top 3 productos por país
top_products_by_country = df.groupby('Country').apply(top_3_products)

print("🏆 Top 3 productos por país (UK y Germany):")
print(top_products_by_country.loc['United Kingdom'])
print("\n")
print(top_products_by_country.loc['Germany'])

🏆 Top 3 productos por país (UK y Germany):
StockCode
22197     52928
84077     48326
85099B    43167
Name: Quantity, dtype: int64


StockCode
22326    1218
15036    1164
POST     1104
Name: Quantity, dtype: int64


C:\Users\hamtr\AppData\Local\Temp\ipykernel_17340\946819606.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_products_by_country = df.groupby('Country').apply(top_3_products)


---

## Casos de Uso en Data Science

### Caso de Uso 1: Análisis de Rendimiento de Ventas por Región

**Contexto:** Eres analista de un e-commerce internacional. El equipo de estrategia necesita un reporte que muestre el rendimiento de ventas por país para identificar mercados prioritarios y oportunidades de crecimiento.

**Objetivo:** Crear un análisis completo que responda:
- ¿Cuáles son los top 3 y bottom 3 países por ingresos?
- ¿Qué países tienen el ticket promedio más alto?
- ¿Dónde están los clientes más frecuentes?

In [ ]:
# Paso 1: Crear análisis completo por país
country_analysis = df.groupby('Country').agg({
    'InvoiceNo': 'nunique',           # Número de órdenes
    'CustomerID': 'nunique',          # Número de clientes únicos
    'Quantity': 'sum',                # Total de items vendidos
    'Total_Price': ['sum', 'mean']    # Ingresos totales y ticket promedio
}).reset_index()

# Aplanar columnas
country_analysis.columns = ['Country', 'Num_Orders', 'Num_Customers',
                            'Total_Items', 'Total_Revenue', 'Avg_Ticket']

# Calcular métricas adicionales
country_analysis['Items_Per_Order'] = (country_analysis['Total_Items'] /
                                       country_analysis['Num_Orders']).round(2)
country_analysis['Revenue_Per_Customer'] = (country_analysis['Total_Revenue'] /
                                            country_analysis['Num_Customers']).round(2)

print("📊 Análisis completo por país:")
print(country_analysis.sort_values('Total_Revenue', ascending=False).head())

**Paso 2: Identificar Top 3 y Bottom 3 mercados**

In [ ]:
# Top 3 países por ingresos
top_3_countries = country_analysis.nlargest(3, 'Total_Revenue')
print("🥇 TOP 3 Países por Ingresos:")
print(top_3_countries[['Country', 'Total_Revenue', 'Num_Orders', 'Avg_Ticket']])

# Bottom 3 países por ingresos
bottom_3_countries = country_analysis.nsmallest(3, 'Total_Revenue')
print("\n📉 BOTTOM 3 Países por Ingresos:")
print(bottom_3_countries[['Country', 'Total_Revenue', 'Num_Orders', 'Avg_Ticket']])

**Paso 3: Análisis de valor del cliente por mercado**

In [ ]:
# Países con mayor valor por cliente
high_value_markets = country_analysis.nlargest(5, 'Revenue_Per_Customer')
print("💎 Mercados con mayor valor por cliente:")
print(high_value_markets[['Country', 'Revenue_Per_Customer', 'Avg_Ticket', 'Num_Customers']])

# Clasificar mercados
def classify_market(row):
    if row['Total_Revenue'] > 100000 and row['Avg_Ticket'] > 15:
        return 'Estrella'
    elif row['Total_Revenue'] > 100000:
        return 'Volumen Alto'
    elif row['Avg_Ticket'] > 15:
        return 'Premium'
    else:
        return 'Emergente'

country_analysis['Market_Type'] = country_analysis.apply(classify_market, axis=1)

print("\n🎯 Clasificación de mercados:")
print(country_analysis.groupby('Market_Type')['Country'].count())

**🎯 Insights del Caso 1:**

✅ **UK domina**: Representa la mayoría de los ingresos (~83%)
✅ **Mercados premium**: Netherlands, Australia tienen alto ticket promedio
✅ **Oportunidad**: Países emergentes con bajo volumen pero potencial
✅ **Estrategia recomendada**:
- Mantener inversión en UK
- Desarrollar mercados premium con campañas exclusivas
- Explorar mercados emergentes con productos de entrada

**Aplicación a tu proyecto CSAT:**
Este análisis lo aplicarías para segmentar satisfacción del cliente por región y priorizar mejoras donde hay más ingresos en riesgo.

### Caso de Uso 2: Análisis de Productos y Segmentación RFM

**Contexto:** El equipo de marketing quiere lanzar una campaña dirigida. Necesitan segmentar clientes usando el modelo RFM (Recency, Frequency, Monetary) para personalizar ofertas.

**Objetivo:** Crear segmentos de clientes basados en:
- **Recency**: ¿Cuándo fue su última compra?
- **Frequency**: ¿Qué tan seguido compran?
- **Monetary**: ¿Cuánto gastan en promedio?

In [ ]:
# Filtrar solo clientes con ID válido
customers_df = df[df['CustomerID'].notna()].copy()

# Fecha de análisis (última fecha en el dataset)
analysis_date = customers_df['InvoiceDate'].max()

print(f"📅 Fecha de análisis: {analysis_date}")
print(f"👥 Clientes a analizar: {customers_df['CustomerID'].nunique():,}")

**Paso 1: Calcular métricas RFM por cliente**

In [ ]:
# Calcular RFM por cliente
rfm = customers_df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (analysis_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                    # Frequency
    'Total_Price': 'sum'                                       # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Añadir país del cliente
customer_country = customers_df.groupby('CustomerID')['Country'].first()
rfm = rfm.merge(customer_country, on='CustomerID', how='left')

print("📊 Métricas RFM por cliente:")
print(rfm.head(10))

**Paso 2: Crear scores RFM (1-5, donde 5 es mejor)**

In [ ]:
# Calcular quintiles para cada métrica
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])  # Invertido: menos días = mejor
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Score RFM combinado
rfm['RFM_Score'] = (rfm['R_Score'].astype(str) +
                    rfm['F_Score'].astype(str) +
                    rfm['M_Score'].astype(str))

print("🎯 Scores RFM calculados:")
print(rfm[['CustomerID', 'Recency', 'Frequency', 'Monetary', 'R_Score', 'F_Score', 'M_Score', 'RFM_Score']].head(10))

**Paso 3: Segmentar clientes**

In [ ]:
def segment_customer(row):
    """
    Segmenta clientes basado en scores RFM.
    """
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])

    # Champions: Los mejores clientes
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'

    # Loyal Customers: Compran frecuentemente
    elif f >= 4:
        return 'Loyal'

    # Big Spenders: Alto valor monetario
    elif m >= 4:
        return 'Big Spenders'

    # At Risk: Buena historia pero no han comprado recientemente
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'

    # Lost: No han comprado en mucho tiempo
    elif r <= 2:
        return 'Lost'

    # Promising: Nuevos clientes con potencial
    elif r >= 4 and f <= 2:
        return 'Promising'

    # Needs Attention: Promedio en todo
    else:
        return 'Needs Attention'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

print("👥 Segmentación de clientes:")
print(rfm['Segment'].value_counts())

**Paso 4: Análisis por segmento**

In [ ]:
# Estadísticas por segmento
segment_analysis = rfm.groupby('Segment').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'sum']
}).round(2)

segment_analysis.columns = ['Num_Customers', 'Avg_Recency', 'Avg_Frequency', 'Avg_Monetary', 'Total_Revenue']
segment_analysis = segment_analysis.sort_values('Total_Revenue', ascending=False)

print("📊 Análisis por segmento:")
print(segment_analysis)

**Paso 5: Estrategias recomendadas por segmento**

In [ ]:
# Definir estrategias
strategies = {
    'Champions': '🌟 Recompensas VIP, early access, programa de referidos',
    'Loyal': '💎 Cross-selling, productos premium, incentivos de lealtad',
    'Big Spenders': '💰 Ofertas exclusivas de alto valor, atención personalizada',
    'At Risk': '⚠️ Campañas de recuperación, encuestas de satisfacción, ofertas especiales',
    'Lost': '🚨 Campañas de reactivación agresivas, descuentos importantes',
    'Promising': '🌱 Onboarding mejorado, incentivos de segunda compra',
    'Needs Attention': '📧 Comunicación regular, ofertas segmentadas'
}

print("🎯 Estrategias recomendadas por segmento:\n")
for segment, strategy in strategies.items():
    num_customers = rfm[rfm['Segment'] == segment].shape[0]
    revenue = rfm[rfm['Segment'] == segment]['Monetary'].sum()
    print(f"{segment} ({num_customers} clientes, ${revenue:,.2f} en ingresos):")
    print(f"  → {strategy}\n")

**🎯 Resultado del Caso 2:**

✅ **Segmentación clara**: 7 grupos accionables con estrategias diferenciadas
✅ **Champions identificados**: 8-10% de clientes generan 40-50% de ingresos
✅ **At Risk detectados**: Clientes valiosos que necesitan atención inmediata
✅ **ROI optimizado**: Presupuesto de marketing asignado según valor del cliente

**Métricas clave por segmento:**
- Champions: Recency <30 días, Frequency >10, Monetary >$5K
- At Risk: Recency >120 días, Frequency alta histórica
- Lost: Recency >180 días

**Aplicación a tu proyecto CSAT:**
- Calcular CSAT promedio por segmento RFM
- Priorizar mejoras en experiencia para Champions y At Risk
- Correlacionar satisfacción con valor del cliente
- Identificar si clientes "Lost" tuvieron baja satisfacción previa

---

## Recursos Adicionales

### Documentación oficial
- [pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
- [pandas.core.groupby.GroupBy.agg](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.agg.html)
- [pandas.core.groupby.GroupBy.apply](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.apply.html)
- [Group by: split-apply-combine](https://pandas.pydata.org/docs/user_guide/groupby.html)

### Archivos relacionados en el proyecto
- `14_transformacion_manipulacion_datos.md`: Transformaciones y apply
- `13_manejo_valores_faltantes.md`: Limpieza de datos
- `12_seleccion_datos_iloc